In [1]:
import numpy as np
import sympy as sym
import numba
import pydae.build as db

### Electromechanical differential equations
\begin{eqnarray}
f_1 &=& \dot \delta = \Omega_b \left( \omega - \omega_s  \right) \\
f_2 &=& \dot \omega = \frac{1}{2H} \left( p_m - p_e - D   \left( \omega - \omega_s \right) \right)   
\end{eqnarray}

### Electric rotor differential equations
\begin{eqnarray}
f_3 &=& \dot e_q' =  \frac{1}{T'_{d0}} \left( -e'_q - \left(X_d - X'_d \right) i_d + v_f^\star \right) \\
f_4 &=& \dot e'_d = \frac{1}{T'_{q0}} \left( -e'_d - \left(X_q - X'_q \right) i_q \right)       
\end{eqnarray}


### Park transform
\begin{eqnarray}
v_d &=& v_t \sin\left(\delta - \theta_t\right) \\
v_q &=& v_t \cos\left(\delta - \theta_t\right) \\
p_e &=& \left( v_q + R_a i_q  \right) i_q + \left( v_d + R_a i_d  \right) i_d  
\end{eqnarray}

### Stator equations
\begin{eqnarray}
g_1 &=& v_q + R_a i_q + X'_d i_d - e'_q\\
g_2 &=& v_d + R_a i_d - X'_q i_q - e'_d\\
\end{eqnarray}

### Powers
\begin{eqnarray}
g_3 &=&  i_d v_d + i_q v_q - p_t \\
g_4 &=&  i_d v_q - i_q v_d - q_t
\end{eqnarray}

### Network equations
\begin{eqnarray}
g_5 &=&   p_t - \left(v_t v_0 \sin\left(\theta_t - \theta_0\right)\right)/X_l\\
g_6 &=&   q_t + \left(v_t v_0 \cos\left(\theta_t - \theta_0\right)\right)/X_l - v_t^2/X_l \\
g_7 &=&  p_l - \left(v_0 v_t \sin\left(\theta_0 - \theta_t\right)\right)/X_l\\
g_8 &=&  q_l + \left(v_0 v_t \cos\left(\theta_0 - \theta_t\right)\right)/X_l - v_0^2/X_l
\end{eqnarray}    



## System definition 

In [23]:
params_dict = {'X_d':1.81,'X1d':0.3, 'T1d0':8.0,  # synnchronous machine d-axis parameters
               'X_q':1.76,'X1q':0.65,'T1q0':1.0,  # synnchronous machine q-axis parameters
               'R_a':0.003,
               'X_l': 0.02, 
               'H':3.5,'D':0.01,
               'Omega_b':2*np.pi*50,'omega_s':1.0,
               'K_delta':0.1,'K_v':1e-3
               }

u_ini_dict = {'v_t':0.8,'theta_t':1.0,'p_l':0.0,'q_l':0.0}  # for the initialization problem
u_run_dict = {'p_m':0.8,'v_f':1.0,'p_l':0.0,'q_l':0.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['delta','omega','e1q','e1d']    # dynamic states
y_ini_list = ['i_d','i_q','p_t','q_t','p_m','v_f','v_0','theta_0']   
y_run_list = ['i_d','i_q','p_t','q_t','v_t','theta_t','v_0','theta_0']   

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [24]:
# auxiliar equations
v_d = v_t*sin(delta - theta_t)  # park
v_q = v_t*cos(delta - theta_t)  # park

p_e = i_d*(v_d + R_a*i_d) + i_q*(v_q + R_a*i_q) # electromagnetic power

# dynamic equations
ddelta = Omega_b*(omega - omega_s) - K_delta*delta  # load angle
domega = 1/(2*H)*(p_m - p_e - D*(omega - omega_s)) # speed
de1q = 1/T1d0*(-e1q - (X_d - X1d)*i_d + v_f + K_v*(1-v_t))
de1d = 1/T1q0*(-e1d + (X_q - X1q)*i_q)

# algrbraic equations
g_1 = v_q + R_a*i_q + X1d*i_d - e1q # stator
g_2 = v_d + R_a*i_d - X1q*i_q - e1d # stator
g_3 = i_d*v_d + i_q*v_q - p_t # active power 
g_4 = i_d*v_q - i_q*v_d - q_t # reactive power
g_5 = p_t - (v_t*v_0*sin(theta_t - theta_0))/X_l  # network equation (p)
g_6 = q_t + (v_t*v_0*cos(theta_t - theta_0))/X_l - v_t**2/X_l  # network equation (q) 
g_7 = -p_l - (v_t*v_0*sin(theta_0 - theta_t))/X_l  # network equation (p)
g_8 = -q_l + (v_t*v_0*cos(theta_0 - theta_t))/X_l - v_0**2/X_l  # network equation (q)

In [25]:
sys = {'name':'iso_milano_ex8p1_4ord_uctrl',
       'params_dict':params,
       'f_list':[ddelta,domega,de1q,de1d],
       'g_list':[g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8],
       'x_list':x_list,
       'y_ini_list':y_ini_list,
       'y_run_list':y_run_list,
       'u_ini_dict':u_ini_dict,
       'u_run_dict':u_run_dict,
       'h_dict':{'p_m':p_m,'p_e':p_e, 'v_f':v_f}}

sys = db.system(sys)
db.sys2num(sys)

In [11]:
u_ini_dict

{'v_t': 0.8, 'theta_t': 1.0, 'p_l': 0.0, 'q_l': 0.0}

In [12]:
u_ini_dict

{'v_t': 0.8, 'theta_t': 1.0, 'p_l': 0.0, 'q_l': 0.0}